# Summary
In this code, contrast adjustment was applied to the dataset in addition to EfficientNet B3. <br>
It was found that the score was improved compared to just EfficientNet B3 in my previous model after only 10 epochs.<br>
<br>
Ref.: I learned a lot from the following notebook for EfficientNet model. I would like to extend my appreciation to the author. <br>
>    Cassava Disease 🌿 Keras + TF + EfficentNet (90%)<br>
>    https://www.kaggle.com/andreshg/cassava-disease-keras-tf-efficentnet-90


# Import necessary modules

In [ ]:
import imp
import random
import warnings
from keras import layers, models
from keras import Model
from keras import callbacks
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import load_model
from tensorflow.python.training.tracking import base
from sklearn.model_selection import train_test_split

# Data frame Preparation

In [ ]:
train_csv_location = '../input/cassava-leaf-disease-classification/train.csv'
train_image_location = '../input/cassava-leaf-disease-classification/train_images'
test_image_location = '../input/cassava-leaf-disease-classification/test_images'

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 21
seed_everything(seed)
warnings.filterwarnings('ignore')

traindf = pd.read_csv(train_csv_location, dtype=str)
test_image_dir = os.listdir(test_image_location)
testdf = pd.DataFrame(test_image_dir, columns=['image_id'])

train, valid = train_test_split(traindf, test_size = 0.05, 
    random_state = 42, stratify = traindf['label']
    )

IMG_SIZE = 456
size = (IMG_SIZE,IMG_SIZE)
n_CLASS = 5
BATCH_SIZE = 15

# Image Contrast Change
This changes contrast of images. It looks to me that disease pattern on the leaf became clearer.<br>


In [ ]:
def adj_contrast(img):
    img_adj = tf.image.adjust_contrast(img,2)
    return img_adj

> # Example (before contrast adjustment)

In [ ]:
import cv2
img = cv2.imread('../input/cassava-leaf-disease-classification/train_images/1000201771.jpg')
plt.imshow(img)

# Example (after contrast adjustment)
Contrast adjustment makes pattern on leaves clearer.

In [ ]:
img_adj=adj_contrast(img)
print('After contrast adjustment')
plt.imshow(img_adj)

# Image Generator, Augmentation, Models, Submission
For preprocessing_function in ImageDataGenerator, I used the defined "adj_contrast" which changes contrast of images.<br>
I imported a model from my private file. To make a model based on EfficientNetB3, please refer to the following. <br>
>    Cassava Disease 🌿 Keras + TF + EfficentNet (90%)<br>
>    https://www.kaggle.com/andreshg/cassava-disease-keras-tf-efficentnet-90

In [ ]:
datagen_train = ImageDataGenerator(
    preprocessing_function= adj_contrast,
    rotation_range=40,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    vertical_flip= True,
    fill_mode='nearest'
)
datagen_val = ImageDataGenerator(
    preprocessing_function= adj_contrast,
)
test_datagen=ImageDataGenerator(
    preprocessing_function= adj_contrast,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe = train,
    directory = train_image_location,
    seed= 42,
    x_col = 'image_id',
    y_col = 'label',
    target_size= size,
    class_mode= 'categorical',
    interpolation= 'nearest',
    shuffle= True,
    batch_size= BATCH_SIZE
)

valid_generator = datagen_val.flow_from_dataframe(
    dataframe = valid,
    directory= train_image_location,
    seed= 42,
    x_col = 'image_id',
    y_col = 'label',
    target_size= size,
    class_mode= 'categorical',
    interpolation= 'nearest',
    shuffle= True,
    batch_size= BATCH_SIZE
)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory= test_image_location,
    seed=42,
    x_col = 'image_id',
    y_col = None,
    target_size= size,
    interpolation= 'nearest',
    shuffle= False,
    batch_size=1,
    class_mode=None,
)

EPOCHS = 10
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

es = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    patience=3,
    restore_best_weights=True, 
    verbose=1,
)

checkpoint_cb = ModelCheckpoint(
    "best_model7_2x.h5",
    save_best_only=True,
    monitor='val_loss',
    mode='min',
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-6,
    mode='min',
    verbose=1,
)

# Fit the model
model = load_model('../input/cassavaleafdiseaseclassification/best_model7_2x.h5')
history = model.fit(
    train_generator, 
    validation_data=valid_generator,                
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_steps=STEP_SIZE_VALID,
    callbacks=[es,checkpoint_cb, reduce_lr],
    workers=16,
)
model.save('./best_model7_2.h5')

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from tensorflow.compat.v1.keras import backend as K
K.set_session(sess)

final_model = models.load_model('./best_model7_2x.h5')

# predict output
test_generator.reset()
pred = final_model.predict(test_generator, steps = STEP_SIZE_TEST, verbose = 1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

# export to csv
filenames=test_generator.filenames
results=pd.DataFrame({"image_id":filenames, "label":predictions})
results.to_csv("submission.csv",index=False)